Norvig 建议在有一等函数的语言重新审视 “策略” “命令” “模板方法” 和 “访问者” 模式。通常，我们我们可以把这些模式中涉及的某些类的实例替换成简单的函数，从而减少样板代码，本章将用函数对象重构 “策略” 模式

策略模式就是定义一些列算法，把它们意义封装起来，并且使它们可以相互替换。本模式使得算法可以独立于它的客户而变化

电商领域有个功能可以明显使用 “策略” 模式，即根据客户的属性或订单中的商品计算折扣，例如有个网店有如下折扣规则

- 有 1000 或以上积分的顾客，每个订单享 %5 折扣。
- 同一订单中，单个商品数量达到 20 或以上，享 %10 折扣
- 订单中的不同商品达到 10 个或以上，享 %7 折扣

简单起见，我们规定一个订单只能享有一种折扣。策略涉及如下几个概念:

上下文：

- 把一些计算委托给实现不同算法的可互换组件，它提供服务。在这个电商例子中，上下文是 Order，它会根据不同的算法计算促销折扣

策略：

- 实现不同算法组件的具体接口。在这个例子中，名为 Promotion 的抽象基类扮演这个角色

具体策略：

- “策略” 的具体子类。fidelityPromo, BulkPromo, LargeOrderPromo 是这里的三个具体策略

下面例子，实例化订单之前，系统会以某种方式选择一种促销折扣策略，然后传给 Order 构造方法。具体怎么选择策略，不在这个模式职责范围内：

In [12]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order: #上下文
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        '''返回折扣金额(正值)'''
        
class FidelityPromo(Promotion):
    '''为积分为 1000 或以上的顾客提供 5% 折扣'''
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion):
    '''单个商品为 20 个或以上时提供 %10 折扣'''
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
    
class LargeOrderPromo(Promotion):
    '''订单中的不同商品达到 10 个或以上时提供 %7 折扣'''
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

Python 3.4 中，声明抽象基类的最简单方式是子类化 abc.ABC。我们的例子就是这么做的，从 Python 3.0 到 Python 3.3，必须在 class 语句使用 metaclas = 关键字

使用不同促销折扣的 Order 类实例：

In [13]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100) # joe 积分是 0，ann 积分是 1100

cart = [LineItem('banana', 4, .5),  #有 3 个商品购物车
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo()) # fidelityPromo 没有给 joe 提供折扣

<Order total: 42.00 due 42.00>

In [14]:
Order(ann, cart, FidelityPromo()) # ann 得到了 5% 的折扣，因为他积分大于 1000

<Order total: 42.00 due 39.90>

In [15]:
banana_cart = [LineItem('banana', 30, .5),  # 30 个香蕉和 10 个苹果
              LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo()) # BulkItemPromo 为 joe 购买的香蕉优惠了 1.5 元

<Order total: 30.00 due 28.50>

In [16]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]  # 构造 10 种不同的商品
Order(joe, long_order, LargeOrderPromo()) # LargeOrderPromo 为 joe 的整个订单提供了 7% 的折扣

<Order total: 10.00 due 9.30>

In [17]:
Order(joe, cart, LargeOrderPromo()) # 

<Order total: 42.00 due 42.00>

## 用函数实现 “策略” 模式

在上面，每个具体的策略都是一个类，而且都只定义了一个方法，即 discount。此外，策略实例没有状态（没有实例属性）。它们看起来就像普通的函数，所以我们可以考虑把上面的策略换成简单的函数，并去掉 Promo 抽象基类

In [ ]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order: #上下文
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self) # 这里修改了直接调用函数
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due {:.2f}>'
        return fmt.format(self.total(), self.due())

        
def fidelity_promo(order):
    '''为积分为 1000 或以上的顾客提供 5% 折扣'''
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
def bulk_item_promo(order):
    '''单个商品为 20 个或以上时提供 %10 折扣'''
    
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount
    
def large_order_promo(order):
    '''订单中的不同商品达到 10 个或以上时提供 %7 折扣'''
    
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

新的 Order 类使用更加简单，如下所示：

In [23]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100) # joe 积分是 0，ann 积分是 1100

cart = [LineItem('banana', 4, .5),  #有 3 个商品购物车
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, fidelity_promo) # fidelityPromo 没有给 joe 提供折扣

<Order total: 42.00 due 42.00>

In [24]:
Order(ann, cart, fidelity_promo) # ann 得到了 5% 的折扣，因为他积分大于 1000

<Order total: 42.00 due 39.90>

In [25]:
banana_cart = [LineItem('banana', 30, .5),  # 30 个香蕉和 10 个苹果
              LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo) # BulkItemPromo 为 joe 购买的香蕉优惠了 1.5 元

<Order total: 30.00 due 28.50>

In [26]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]  # 构造 10 种不同的商品
Order(joe, long_order, large_order_promo) # LargeOrderPromo 为 joe 的整个订单提供了 7% 的折扣

<Order total: 10.00 due 9.30>

In [27]:
Order(joe, cart, large_order_promo) # 

<Order total: 42.00 due 42.00>

看到测试结果一样，并且简单了一些

## 选择最佳策略

我们现在想每次选出来的策略是最省钱的，可以简单的写成如下：

In [28]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)

In [29]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due 9.30>

In [30]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due 28.50>

In [31]:
Order(ann, cart, best_promo)

<Order total: 42.00 due 39.90>

上面例子虽然可用且容易阅读，但是有些重复可能导致不易察觉的缺陷，若想添加新的促销策略，不仅要定义相应的函数，而且还要把它添到 promos 列表，我们会在接下来改进它

## 找出模块中的全部策略

在 Python 中，模块也是一等对象，而且标准库中提供了几个处理模块的函数。Python 文档是这样说明内置函数 globals 的

globals()

返回一个字典，表示当前全局符号表。这个符号表始终针对当前模块（对函数或方法来说，是指定义它们的模块，而不是调用它们的模块）

下面使用 globals 函数帮助 best_promo 自动找到其它可用的 `*_promo` 函数：

In [32]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']
def best_promo(order):
    '''选择可用的最佳折扣'''
    return max(promo(order) for promo in promos)

另一个可行的方法是将所有的策略函数都存放在一个单独的模块中，除了 best_promo，这里我们将 3 个策略函数存放在了 promotions.py 中

下面的代码中，最大的变化时内省名为 promotions 的独立模块，构建策略函数列表。注意，下面要导入 promotions 模块，以及高阶内省函数的 inspect 模块（简单起见，这里没有给出导入语句，因为导入语句一般在文件顶部）

In [1]:
import inspect
import promotions

promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]
print(promos)
def best_promo(order):
    '''选择最佳折扣'''
    return max(promo(order) for promo in promos)

[<function bulk_item_promo at 0x7ff2845dbb70>, <function fidelity_promo at 0x7ff2886540d0>, <function large_order_promo at 0x7ff2845dbea0>, <function namedtuple at 0x7ff290e38a60>]


inspect.getmembers 函数用户获取对象（这里是 promotions 模块）的属性，第二个参数是可选的判断条件（一个布尔值函数）。我们使用的是 inspect.isfunction，只获取模块中的函数

当然这依赖我们将所有的策略都放到 promotions 模块中，如果有人把别的函数放到这里，我们的代码可能会报错，所以可以考虑做更严格测筛选。不过动态收集促销折扣函数更为显式的一种方案是使用简单的装饰器。在第 7 章讨论函数装饰器时会使用其它方式实现这个电商 “策略” 模式示例

## “命令” 模式

命令模式的目的是解耦调用操作的对象（调用者）和提供实现的对象（接收者）。在《设计模式：可复用面向对象软件的基础》所举示例中，调用者是图形应用程序的菜单项，而接收者是被编辑的文档或应用程序自身。

这个模式做法是，在二者之间放一个 Command 对象，让它实现只有一个方法（execute）的接口，调用接收者中的方法执行所需的操作。这样，调用者无需了解接收者的接口，而且不同的接收者可以使用不同的 Command 子类。调用者有一个具体的命令，通过调用 execute 方法执行。

Gamma 等人说过，”命令模式是回调机制面向对象的替代品“，问题是，我们有时候不需要回调机制的面向对象替代品。

我们可以不为调用者提供一个 Command 实例，而是给他一个函数，此时，调用者不用调用 command.execute()，直接调用 command() 即可。MacroCommand 可以实现成定义了 `__call__` 方法的类。这样， MacroCommand  的实例就是可调用对象，各自维护着一个函数列表，供以后调用。

In [3]:
class MacroCommand:
    '''一个执行一组命令的命令'''
    
    def __init__(self, commands):
        self.commands = list(commands) # 1
    
    def __call__(self):
        for command in self.commands: # 2
            command()

1. 使用 commands 参数构建一个列表，既能确保参数是可迭代对象，还能在各个 MacroCommand 实例中保存各个命令引用的副本
2. 调用 MacroCommand 实例时，self.command() 中的各个命令按顺序执行

复杂的 ”命令“ 模式（比如说支持撤销操作）可能需要更多，而不仅仅是简单的灰调函数，即使如此，也可以考虑使用 Python 提供的几个替代品

- 像上面例子中 MacroCommand 那样的可调用实例，可以保存任何所需状态，而且除了 `__call__` 之外还可以提供其它方法
- 可以使用闭包在调用之间保存函数的内部状态